 * Copyright 2024 Xue_Lexiang
 * Licensed under MIT (https://github.com/xuelx1/LearnQC/LISENCE)

### Modular Addition and Multiplication

#### 1.Modular Adder Gate

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT 
import numpy as np
import math
import itertools

backend = AerSimulator()

In [3]:
def getAngles(a, N):
    s = bin(int(a))[2:].zfill(N)
    angles = np.zeros(N)
    for i in range(N):
        for j in range(i, N):
            if s[j] == '1':
                angles[N-i-1] += math.pow(2, i-j)
        angles[N-1-i] *= math.pi    
    return angles


In [4]:
def phiADD(a, N):
    qc = QuantumCircuit(N)
    angels = getAngles(a, N)
    for i in range(N):
        qc.p(angels[i], i)    
    qc = qc.to_gate()
    qc.name = "phiADD %i" % (a)
    return qc

def cphiADD(a, N):
    qc = QuantumCircuit(N+1)
    angels = getAngles(a, N)
    for i in range(N):
        qc.cp(angels[i], 0, i+1)    
    qc = qc.to_gate()
    qc.name = "cphiADD %i" % (a)
    return qc

def ccphiADD(a, N):
    qc = QuantumCircuit(N+2)
    angle=getAngles(a, N)
    for i in range(N):
        qc.mcp(angle[i], [0, 1], i+2)
    qc = qc.to_gate()
    qc.name = "ccphiADD %i" % (a)
    return qc

In [5]:
def ccphiADDmodN(a, n, N):
    '''a N+3-qubit gate, first two qubits are the control
    from 3 to N+2 are qubits, which receive phase of b and return phase of a+b mod n
    the last qubit is the ancilla'''
    qft = QFT(N, do_swaps=False)
    qft_inv = qft.inverse()
    ccadd_a = ccphiADD(a, N)
    ccadd_a_inv = ccadd_a.inverse()
    add_n = phiADD(n, N)
    add_n_inv = add_n.inverse()
    cadd_n = cphiADD(n, N)
    qc = QuantumCircuit(N+3)
    
    qc.append(ccadd_a, list(range(N+2)))
    qc.append(add_n_inv, list(range(2, N+2)))
    qc.append(qft_inv, list(range(2, N+2)))
    qc.cx(N+1, N+2)
    qc.append(qft, list(range(2, N+2)))
    qc.append(cadd_n, [N+2] + list(range(2, N+2))) 
    qc.append(ccadd_a_inv, list(range(N+2)))
    qc.append(qft_inv, list(range(2, N+2)))
    qc.x(N+1)
    qc.cx(N+1, N+2)
    qc.x(N+1)
    qc.append(qft, list(range(2, N+2)))
    qc.append(ccadd_a, list(range(N+2)))
    return qc


In [12]:
n = 8
N = math.ceil(math.log2(n))
a = 7
b = 5
qc = QuantumCircuit(N+3)
qft = QFT(N, do_swaps=False)
qft_inv = qft.inverse()
bin_b = bin(b)[2:].zfill(N)
qc.x([0, 1])
for i in range(N):
    if bin_b[i] == '1':
        qc.x(i+2)
qc.append(qft, list(range(2, N+2)))
modadder_circuit = ccphiADDmodN(a, n, N)
qc.append(modadder_circuit, list(range(N+3)))
qc.append(qft_inv, list(range(2, N+2)))
qc = transpile(qc, backend)
qc.measure_all()
job = backend.run(transpile(qc, backend), shots=1024)
res_dict = job.result().get_counts()
print(res_dict)


{'100011': 1024}


In [7]:
n = 7
N = math.ceil(math.log2(n))
for l in itertools.product(range(7), repeat=2):
    a_bin = bin(l[0])[2:].zfill(N)
    b_bin = bin(l[1])[2:].zfill(N)
    qc = QuantumCircuit(N+3)
    qft = QFT(N, do_swaps=False)
    qft_inv = qft.inverse()
    qc.x([0, 1])
    for i in range(N):
        if bin_b[i] == '1':
            qc.x(i+2)
    qc.append(qft, list(range(2, N+2)))
    modadder_circuit = ccphiADDmodN(l[0], n, N)
    qc.append(modadder_circuit, list(range(N+3)))
    qc.append(qft_inv, list(range(2, N+2)))
    qc.measure_all()
    job = backend.run(transpile(qc, backend), shots=1024)
    res_str = job.result().get_counts().most_frequent()
    print(a_bin, b_bin, res_str)

000 000 111011
000 001 111011
000 010 111011
000 011 111011
000 100 111011
000 101 111011
000 110 111011
001 000 000011
001 001 000011
001 010 000011
001 011 000011
001 100 000011
001 101 000011
001 110 000011
010 000 000111
010 001 000111
010 010 000111
010 011 000111
010 100 000111
010 101 000111
010 110 000111
011 000 001011
011 001 001011
011 010 001011
011 011 001011
011 100 001011
011 101 001011
011 110 001011
100 000 001111
100 001 001111
100 010 001111
100 011 001111
100 100 001111
100 101 001111
100 110 001111
101 000 101111
101 001 101111
101 010 101111
101 011 101111
101 100 101111
101 101 101111
101 110 101111
110 000 110011
110 001 110011
110 010 110011
110 011 110011
110 100 110011
110 101 110011
110 110 110011


In [8]:
# test adder
#n = 7
#N = math.ceil(math.log2(n))

#qc = QuantumCircuit(N+3)
#qft_inv = QFT(N, do_swaps=False, inverse=True)
#for a in range(n):
#    for b in range(n):
#        bin_b = bin(b)[2:].zfill(N)
#        for i in range(N):
#            if bin_b[i] == '1':
#                qc.x(i+2)
#        qc.h(list(range(2, N+2)))
#       modadder_circuit = ccphiADDmodN(a, n, N)
#        qc.append(modadder_circuit, list(range(N+3)))
#        qc.append(qft_inv, list(range(2, N+2)))
#        qc.measure_all()
#        job = backend.run(transpile(qc, backend), shots=1024)
#        res_dict = job.result().get_counts()
#        res = res_dict.most_frequent()[2:N+3]
#        #res_val = 0
#        #for i in range(N):
#        #    if res[i] == '1':
#        #        res_val += 2**(N-1-i)
#        #print(f"{a} + {b} = {res_val}")
#        print(res)                